Here we implement the solver from [Quantum algorithm for solving linear differential equations: Theory and experiment](https://journals.aps.org/pra/pdf/10.1103/PhysRevA.101.032307) by Xin *et al.* in the **Classiq** quantum programming language. 

### Classical Problem

Xin *et al.* provide a new method to solve any Linear Differential Equation formulated as 

$$\frac{d}{dt}\vec{x}(t)= M\vec{x}(t)+\vec{b}$$

for an $N\times N$ matrix $M$ and $N$-dimensional vectors $x$, $b$. 

Taylor-expanding the analytical solution, 

$$
\begin{aligned}
x(t)&=e^{Mt}x(0)+\left(e^{Mt}-\mathbb{I}\right)M^{-1}b
\\
&\approx \sum^k_{m=0}\frac{(Mt)^m}{m!}x(0)+\sum^k_{n=1}\frac{M^{n-1}t^n}{n!}b.
\end{aligned}
$$

### Quantum Formulation

1. Prepare two quantum states $\ket{x(0)}$ and $\ket{b}$ in the $N$-dimensional computational basis, on $\log_2(N)$ qubits. 
1. Describe the $N\times N$ operator $M$ as $A$ in the computational basis, such that $A_{ij}=M_{ij}\ket{i}\bra{j}$ up to normalization.

We then arrive at 

$$\ket{x(t)}\approx \sum_{m=0}^k \frac{\|x(0)\|(\|M\|At)^m}{m!}\ket{x(0)}+\sum_{n=1}^k\frac{\|b\|(\|M\|A)^{n-1}t^n}{n!}\ket{b}$$


#### More Encodings
1. Assuming $A$ is unitary, encode $k$ powers of $A$ as the unitaries $U_n=A^n$.
1. Define $C_m=\|x(0)\|(\|M\|t)^m/m!$ and $D_n=\|b\|(\|M\|t)^{n-1}t/n!$



Import all relevant Classiq modules and functions! You need to install `classiq` from `pip` beforehand. You can do this with Anaconda via `conda env create -f environment.yml`. 

In [ ]:
# ONLY RUN THIS ONCE TO AUTHENTICATE DEVICE
import classiq
classiq.authenticate(overwrite=True)

In [1]:
import numpy as np
np.set_printoptions(precision=3)
import scipy.linalg as la
import math
from util import quantum_encode, is_unitary
from Code_For_V_and_W_Operators import V_operator, normalization, construct_VS1_VS2, construct_W_WS1_WS2
import time

from classiq import (
    qfunc, Output, QBit, QNum, QArray, CInt,
    allocate, within_apply, control, repeat, if_, 
    inplace_prepare_state, hadamard_transform, inplace_prepare_int, 
    inplace_prepare_amplitudes, show, 
    create_model, write_qmod,
    X, H, SWAP, unitary,
    Constraints, Preferences, 
)
from classiq.synthesis import synthesize, set_constraints, set_execution_preferences, set_preferences
from classiq.executor import execute
from classiq.execution import ExecutionPreferences, IBMBackendPreferences,  IBMBackendProvider
from classiq.interface.generator.quantum_program import QuantumProgram

def write_qprog(qprog, fname):
    # classiq doesn't have a built-in way to write a qprog to a file for some reason
    file = open(fname,"w")
    file.write(qprog)
    file.close()

To submit the code to Classiq and process it, we wrap all of the native `qfunc`s with a larger function that allows us to vary the order of the Taylor approximation $k$, the time $t$, and the error bound of the amplitude encoding.

In [4]:
class QuantumLDESolver():
    """
    Class of quantum algorithms implemented from "Quantum algorithm for solving 
     linear differential equations: Theory and experiment" by Xin et. al. (2020) 
     at https://journals.aps.org/pra/pdf/10.1103/PhysRevA.101.032307

     TODO: implement non-unitary M solving
    """

    def __init__(self, M, x0, b, k=2, err_bound=0.1):
        """
        Initialize a single LDE problem of dx/dt = Mx + b
        """
        self.k = k
        self.err_bound = err_bound

        self.x0 = quantum_encode(x0)
        self.b  = quantum_encode(b)
        if len(x0) != len(b):
            raise ValueError("x0 and b must be the same length")
        self.M = np.array(M)
        if len(x0) < len(M[0]):
            raise ValueError("M is too big for given x0, b")
        if len(x0) != len(M[0]):
            # extend M to larger hilbert space of x0
            newM = np.identity((len(x0)))
            newM[:len(M[0]),:len(M[0])] = M
            self.M = newM
        
        
        # compute all norms once
        self.x0_norm = np.linalg.norm(x0)
        self.b_norm = np.linalg.norm(b)
        # use order-2 norm to preserve unitarity
        self.M_norm = np.linalg.norm(M, ord=2)
        A = M / self.M_norm
        self.isUnitary = is_unitary(A)
        if not self.isUnitary:
            raise NotImplementedError("Non-unitary M not yet implemented")
        self.U = [np.linalg.matrix_power(A, i).tolist() for i in range(0,k+1)]
        nqubits = math.ceil(math.log2(len(x0))) + math.ceil(math.log2(self.k+1))

        nqubits += 1 if self.b_norm != 0 else 0
        print(f'Needs at least {nqubits} qubits to run!')
    #
    # HELPER FUNCTIONS
    #
    def get_Cs_and_Ds(self, M_norm, x0_norm, b_norm, k, t):
        ''' compute C and D vecs when b is nonzero '''
        C, D = np.empty(k+1), np.empty(k+1)
        coeff = 1
        for i in range(k+1):
            # D[0] = 0, reuse last coefficient
            if i != 0:
                D[i-1] = b_norm * coeff * t / i
                coeff *= M_norm * t / i
            C[i] = x0_norm * coeff
        
        Csum, Dsum = np.sqrt(np.sum(C)), np.sqrt(np.sum(D))
        #N = np.sqrt(Csum + Dsum)
        C_amps = np.sqrt(C) / Csum
        D_amps = np.sqrt(D) / Dsum
        return Csum, Dsum, C_amps.tolist(), D_amps.tolist()
    
    def get_Cs(self, M_norm, x0_norm, k, t):
        ''' only return C vec when b is zero'''
        C = np.empty(k+1)
        C[0] = x0_norm
        for i in range(1,k+1):
            C[i] = C[i-1] * M_norm * t / i
        # encode amplitudes as probabilities
        Csum = np.sqrt(np.sum(C))
        C_amps = np.sqrt(C) / Csum
        return Csum, C_amps.tolist()

    def inp_multi_reg_amps(self, *args):
        """ prepare multiple QArray amplitudes at once
        Inputs: register1, amps1, register2, amps2, ... registerN, ampsN
        """
        for i in range(0, len(args), 2):
            inplace_prepare_amplitudes(amplitudes=args[i+1], 
                                       target=args[i], bound=self.err_bound)
    #
    # QUANTUM ALGORITHMS in CLASSIQ
    #
    def LDE_no_time_qmod(self, x0, k=2, save_qmod=''):
        """
        t = 0 case, trivially return x0, still in qmod for consistent output
        """
        @qfunc
        def main(work: Output[QArray], taylor: Output[QNum]):
            # allocate all qubits before doing anything
            allocate(math.ceil(math.log2(len(x0))), work)
            allocate(math.ceil(math.log2(k+1)), taylor)
            # encode x0
            self.inp_multi_reg_amps(work, x0)
        
        # create the model!
        qmod = create_model(main)
        # store the .qmod code!
        if save_qmod:
            write_qmod(qmod, save_qmod)
        return qmod

    def unitary_LDE_nob_qmod(self, x0, U, C_amps, k=2, save_qmod=''):
        """ 
        M is unitary AND b = 0 
        """
        @qfunc
        def prepare_registers(work: QArray, taylor: QNum):
            # evolve ancilla into superposition state
            # encode x_0 and Cvals into registers with |0> ancilla
            self.inp_multi_reg_amps(work, x0, taylor, C_amps)

        @qfunc
        def do_entangling(taylor: QNum, work: QArray):
            # apply powers of A to taylor register
            for i in range(k+1):
                control(taylor == i,
                        lambda: unitary(U[i], work))

        @qfunc
        def main(work: Output[QArray], taylor: Output[QNum]):
            # allocate all qubits before doing anything
            allocate(math.ceil(math.log2(len(x0))), work)
            allocate(math.ceil(math.log2(k+1)), taylor)
            # apply V^dag U V
            within_apply(lambda: prepare_registers(work, taylor),
                         lambda: do_entangling(taylor, work))
        
        # create the model!
        qmod = create_model(main)
        # store the .qmod code!
        if save_qmod:
            write_qmod(qmod, save_qmod)
        return qmod
    
    def unitary_LDE_qmod(self, x0, b, U, CD_amp, C_amps, D_amps, k=2,
                       save_qmod=''):
        """ 
        M is unitary AND b != 0
        """
        @qfunc
        def prepare_registers(x0_b_ancilla: QBit, work: QArray, taylor: QNum):
            # evolve ancilla into superposition state
            self.inp_multi_reg_amps(x0_b_ancilla, CD_amp)
            # encode x_0 and Cvals into registers with |0> ancilla
            control(x0_b_ancilla == 0, 
                    lambda: self.inp_multi_reg_amps(work, x0, taylor, C_amps))
            # encode b and Dvals into registers with |1> ancilla
            control(x0_b_ancilla == 1, 
                    lambda: self.inp_multi_reg_amps(work, b, taylor, D_amps))

        @qfunc
        def do_entangling(taylor: QNum, work: QArray):
            # apply powers of A to taylor register
            for i in range(k+1):
                control(taylor == i, 
                        lambda: unitary(U[i], work))

        @qfunc
        def main(x0_b_ancilla: Output[QBit], work: Output[QArray], 
                taylor: Output[QNum]):
            # allocate all qubits before doing anything
            allocate(1, x0_b_ancilla)
            allocate(math.ceil(math.log2(len(x0))), work)
            allocate(math.ceil(math.log2(k+1)), taylor)
            # apply V^dag U V
            within_apply(lambda: prepare_registers(x0_b_ancilla, work, taylor),
                         lambda: do_entangling(taylor, work))
        
        # create the model!
        qmod = create_model(main)
        # store the .qmod code!
        if save_qmod:
            write_qmod(qmod, save_qmod)
        return qmod
    
    def nonunitary_LDE_qmod(self, x0, b, U, CD_amp, C_amps, D_amps, k=2):
        """
        M is non-unitary and b != 0
        """
        return NotImplementedError("Non-unitary M not yet implemented")
    
    def nonunitary_LDE_nob_qmod(self, x0, b, U, C_amps, k=2):
        """
        M is non-unitary and b = 0
        """
        return NotImplementedError("Non-unitary M with zero b yet implemented")
    #
    # CLASSIQ EXECUTION
    #
    def run_qmod(self, qmod, opt='depth', nshots=10000, job_name='',
                 save_qprog='', sim='Classiq',
                 show_circuit=True, print_info=True):
        """ optimize qmod with constraints and run it on a simulator 
         - sim = 'Classiq' or 'IBM', both classical simulators though
        """
        # set constraints and preferences
        if sim == 'Classiq':
            max_width = 25
            qmod = set_preferences(qmod,
                Preferences(backend_service_provider="Classiq",backend_name="simulator"))
        elif sim == 'IBM':
            # REPLACE WITH YOUR OWN IBM API TOKEN!!
            token = '0920786a03ff0a185aca85f5c4a4be13232efe6641aa5685381a1c3984ec1ed1a7fdd85fbf1967af41bebbf30f321eadb4fb7188eb830a91100133a65e9a9fb1'
            max_width = 127
            qmod = set_preferences(qmod,
                IBMBackendPreferences(
                    backend_name="ibm_strasbourg",
                    access_token=token,
                    provider=IBMBackendProvider(hub = "ibm-q", 
                                                group = "open", project = "main")
                )
            )
        
        qmod = set_constraints(qmod,
            Constraints(max_width=max_width, optimization_parameter=opt))
        qmod = set_execution_preferences(qmod,
            ExecutionPreferences(num_shots=nshots, job_name=job_name, 
                                random_seed='767'))
        qmod = set_preferences(qmod,
            Preferences(timeout_seconds=600, optimization_timeout_seconds=120))
    
        # synthesize the circuit
        start_time = time.time()
        if print_info:
            print(f"Running {job_name}")
        qprog = synthesize(qmod)
        QP = QuantumProgram.from_qprog(qprog)
        circuit_width = QP.data.width
        circuit_depth = QP.transpiled_circuit.depth
        end_time = time.time()
        if print_info:
            print(f"\tcircuit synthesized in {end_time-start_time:.2f}s: width={circuit_width},depth={circuit_depth}")
        # open in viewer
        if show_circuit: 
            show(qprog)
        # save generated quantum program
        if save_qprog:
            write_qprog(qprog, save_qprog)
        start_time = time.time()
        
        job = execute(qprog)
        if print_info:
            print(
                f"\tjob with {job.num_shots} shots is {job.status} on provider-backend={job.provider}-{job.backend_name} \n\tand can be accessed at {job.ide_url}"
            )
        results = job.result()[0].value
        end_time = time.time()
        if print_info:
            print(f"\tran in {end_time-start_time:.2f}s")
        return results
    #
    # SOLVING THE WHOLE THING
    #
    def solve(self, times: list[float], opt='depth', nshots=1000, sim='Classiq', open_job=False,
              k = None, err_bound = None):
        # allow rewriting of k and err_bound
        if k is not None:
            self.k = k
        if err_bound is not None:
            self.err_bound = err_bound

        allData = []

        for t in times:
            name = f'k={self.k}_t={round(t,3)}'
            if t == 0:
                qmod = self.LDE_no_time_qmod(self.x0, self.k, save_qmod=f'qmod/{name}')
                N = self.x0_norm
            # generate appropriate quantum circuit
            elif self.isUnitary:
                if self.b_norm == 0:
                    N, C_amps = self.get_Cs(self.M_norm, self.x0_norm, self.k, t)
                    name += '_nob'
                    qmod = self.unitary_LDE_nob_qmod(self.x0, self.U, C_amps, self.k, 
                                                    save_qmod=f'qmod/{name}')
                else:
                    C, D, C_amps, D_amps = self.get_Cs_and_Ds(self.M_norm, self.x0_norm, self.b_norm, self.k, t)
                    N = np.sqrt(C + D)
                    CD_amp = [C/N,D/N]
                    qmod = self.unitary_LDE_qmod(self.x0, self.b, self.U, CD_amp, C_amps, D_amps,
                                        self.k, save_qmod=f'qmod/{name}')
            else:
                return NotImplementedError("Non-unitary M not yet implemented")
            # run the quantum circuit on backend
            ret = self.run_qmod(qmod, opt=opt, nshots=nshots, job_name=name,
                                save_qprog=f'qprog/{name}', sim=sim,
                                show_circuit=open_job, print_info=True)
            allData.append((N,ret))
            # convert results probabilities, using normalization to convert back

            # for now, we store the results dictionary completely, we can always access
            # the data later.

        return allData

### Our Problem
We solve the following differential equation:
$$\frac{d^2y(t)}{dt^2}=y''(t)=-\omega^2 y'(t),\quad y(0)=y'(0)=1\,.$$

We can linearize this in the form $\frac{d}{dt}\mathbf{x} = M\mathbf{x}+\mathbf{b}$ using $x=[y(t),y'(t)]$ to get

$$\frac{d}{dt}\begin{pmatrix}y\\ y'\end{pmatrix} = \begin{bmatrix}0 & 1 \\ -\omega^2 & 0\end{bmatrix}\begin{pmatrix}y\\ y'\end{pmatrix} + \begin{bmatrix}0 \\ 0\end{bmatrix}\,.$$

When $\omega=1$, $M$ is unitary, which allows us to use the easy version of the algorithm that doesn't rely on accessing $L$-level qudits. Furthermore, $b=0$, which means we can ignore the first ancilla qubit and only have to compute half of the Taylor expansion coefficients.

Let's Taylor-expand our solution to 2nd-order and solve for 11 points over $t\in[0,1]$. 

In [5]:
M = np.array([[0,1],[-1,0]])
x0 = [1,1]
b = [0,0]

k = 4
num_timesteps = 11
time_range = (0,1)
times = np.linspace(time_range[0],time_range[1],num_timesteps)
err_bound = 0.01

solver = QuantumLDESolver(M, x0, b, k, err_bound)
results = solver.solve(times, opt='depth', nshots=10000, sim='Classiq', open_job=False)

Needs at least 4 qubits to run!
Running k=4_t=0.0
	circuit synthesized in 2.81s: width=4,depth=1
	job with 10000 shots is QUEUED on provider-backend=Classiq-simulator 
	and can be accessed at https://platform.classiq.io/jobs/1577eac5-be05-4bff-8b6f-81c19feaa354
	ran in 3.03s
Running k=4_t=0.1_nob


ClassiqAPIError: 
	Invalid access to expression log(5)/log(2) as <class 'int'>
		at ipynb cell line 81 character 12 in function inplace_prepare_amplitudes

Error identifier: EDE4F6F99-0BF6-4C13-92ED-BC482E48BEBE.
If you need further assistance, please reach out on our Community Slack channel at: https://short.classiq.io/join-slack

In [ ]:
def all_bitstrings(N):
    """ return strings of all possible N-bit states"""
    ret = []
    for i in range(2**N):
        string = ""
        for n in range(N-1,-1,-1):
            if i >= 2**n:
                string += "1"
                i -= 2**n
            else:
                string += "0"
        ret.append(string)
    return ret

def get_work_strings(nqubits, indices, lsb_right=True):
    """ 
    out of a full state vector nqubits long, generate all possible work qubit states
    which have qubits at indices place.
     - index=0 is the rightmost qubit if lsb_right is True, leftmost if False
    """
    N = len(indices)
    ret = []
    work_bits = all_bitstrings(N)
    for work_bit in work_bits:
        # ancillas are desired to be all zero...
        fullstring = ["0"]*nqubits
        if lsb_right:
            for i in range(N):
                fullstring[nqubits-indices[i]-1] = work_bit[-i]
        else:
            for i in range(N):
                fullstring[indices[i]] = work_bit[-i]
        ret.append(''.join(fullstring))
        # make full string
    return ret

y = np.zeros((len(times), len(x0)))
for i in range(len(times)):
    #nqubits = math.ceil(math.log2(len(x0))) + math.ceil(math.log2(k+1))
    nqubits = 3
    N = results[i][0]  # normalization amount
    ret = results[i][1]  # classiq results dictionary
    total_counts = ret.num_shots
    # reconstruct / parse state vector
    out_map = ret.output_qubits_map   
    vec_indices = out_map['work']         # tuple of indices
    assert 2**len(vec_indices) == len(x0)
    lsb_right = ret.counts_lsb_right  # bool of whether output map starts from right or left of string
    counts_dict = ret.counts
    # set all other qubits to be 0, then reconstruct state vector from counts
    nqubits = len(list(counts_dict.keys())[0])
    work_strings = get_work_strings(nqubits, vec_indices, lsb_right)
    
    # iterate from 000 001 010 011 ...
    for j, work_string in enumerate(work_strings):
        if work_string not in counts_dict:
            print(f"Work string {work_string} not found in counts")
            continue
        y[i,j] = counts_dict[work_string]
    # convert counts to probabilities, then account for total normalization
    y[i,:] = y[i,:] * N**2 / total_counts
    # get final x(t)

import matplotlib.pyplot as plt
def analytical_soln(w, times):
    # x, x'
    return np.array([np.cos(w*times) + np.sin(w*times)/w, -w*np.sin(w*times) + np.cos(w*times)]).T

smooth_times = np.linspace(times[0],times[-1],50)
plt.plot(smooth_times, analytical_soln(1, smooth_times), '--')
print(times.shape, y.shape)
plt.scatter(times, y[:,0], label='$x(t)$')
plt.scatter(times, y[:,1], label="$x'(t)$")
plt.xlabel('time')
plt.ylabel(r'$\vec{x}(t)$')
plt.title('Quantum LDE Solver')